# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 24 2022 10:20AM,237959,10,CLI00282.1,"CLINUVEL, Inc.",Released
1,May 24 2022 10:20AM,237958,10,LOA157337,Beach Products Inc,Released
2,May 24 2022 10:13AM,237957,16,SHL-8056935,"SHL Pharma, LLC",Released
3,May 24 2022 10:11AM,237956,10,0085698119,ISDIN Corporation,Released
4,May 24 2022 10:11AM,237956,10,0085700876,ISDIN Corporation,Released
5,May 24 2022 10:11AM,237956,10,0085700916,ISDIN Corporation,Released
6,May 24 2022 10:11AM,237956,10,0085700918,ISDIN Corporation,Released
7,May 24 2022 10:11AM,237956,10,0085700919,ISDIN Corporation,Released
8,May 24 2022 10:11AM,237956,10,0085700920,ISDIN Corporation,Released
9,May 24 2022 10:11AM,237956,10,0085700927,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
52,237955,Released,49
53,237956,Released,37
54,237957,Released,1
55,237958,Released,1
56,237959,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237955,NaN,NaN,49.0
237956,NaN,NaN,37.0
237957,NaN,NaN,1.0
237958,NaN,NaN,1.0
237959,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237701,0.0,0.0,1.0
237861,0.0,5.0,18.0
237864,0.0,0.0,11.0
237866,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237701,0,0,1
237861,0,5,18
237864,0,0,11
237866,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237701,0,0,1
2,237861,0,5,18
3,237864,0,0,11
4,237866,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237701,,,1
2,237861,,5,18
3,237864,,,11
4,237866,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 24 2022 10:20AM,237959,10,"CLINUVEL, Inc."
1,May 24 2022 10:20AM,237958,10,Beach Products Inc
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC"
3,May 24 2022 10:11AM,237956,10,ISDIN Corporation
40,May 24 2022 10:10AM,237955,10,ISDIN Corporation
89,May 24 2022 10:07AM,237952,10,ISDIN Corporation
124,May 24 2022 10:07AM,237951,10,ISDIN Corporation
159,May 24 2022 10:05AM,237954,10,ISDIN Corporation
163,May 24 2022 10:04AM,237953,10,"Snap Medical Industries, LLC"
167,May 24 2022 10:02AM,237864,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 24 2022 10:20AM,237959,10,"CLINUVEL, Inc.",,,1
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,,,1
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",,,1
3,May 24 2022 10:11AM,237956,10,ISDIN Corporation,,,37
4,May 24 2022 10:10AM,237955,10,ISDIN Corporation,,,49
5,May 24 2022 10:07AM,237952,10,ISDIN Corporation,,,35
6,May 24 2022 10:07AM,237951,10,ISDIN Corporation,,,35
7,May 24 2022 10:05AM,237954,10,ISDIN Corporation,,,4
8,May 24 2022 10:04AM,237953,10,"Snap Medical Industries, LLC",,,4
9,May 24 2022 10:02AM,237864,20,"ACI Healthcare USA, Inc.",,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 10:20AM,237959,10,"CLINUVEL, Inc.",1,,
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,1,,
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",1,,
3,May 24 2022 10:11AM,237956,10,ISDIN Corporation,37,,
4,May 24 2022 10:10AM,237955,10,ISDIN Corporation,49,,
5,May 24 2022 10:07AM,237952,10,ISDIN Corporation,35,,
6,May 24 2022 10:07AM,237951,10,ISDIN Corporation,35,,
7,May 24 2022 10:05AM,237954,10,ISDIN Corporation,4,,
8,May 24 2022 10:04AM,237953,10,"Snap Medical Industries, LLC",4,,
9,May 24 2022 10:02AM,237864,20,"ACI Healthcare USA, Inc.",11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 10:20AM,237959,10,"CLINUVEL, Inc.",1,,
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,1,,
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",1,,
3,May 24 2022 10:11AM,237956,10,ISDIN Corporation,37,,
4,May 24 2022 10:10AM,237955,10,ISDIN Corporation,49,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 10:20AM,237959,10,"CLINUVEL, Inc.",1.0,NaN,NaN
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,1.0,NaN,NaN
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",1.0,NaN,NaN
3,May 24 2022 10:11AM,237956,10,ISDIN Corporation,37.0,NaN,NaN
4,May 24 2022 10:10AM,237955,10,ISDIN Corporation,49.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 10:20AM,237959,10,"CLINUVEL, Inc.",1.0,0.0,0.0
1,May 24 2022 10:20AM,237958,10,Beach Products Inc,1.0,0.0,0.0
2,May 24 2022 10:13AM,237957,16,"SHL Pharma, LLC",1.0,0.0,0.0
3,May 24 2022 10:11AM,237956,10,ISDIN Corporation,37.0,0.0,0.0
4,May 24 2022 10:10AM,237955,10,ISDIN Corporation,49.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3093372,201.0,8.0,0.0
12,237883,0.0,1.0,0.0
15,1427592,57.0,57.0,0.0
16,951659,5.0,8.0,0.0
17,237924,1.0,0.0,0.0
18,951237,20.0,3.0,0.0
19,1427549,5.0,7.0,0.0
20,1665277,87.0,30.0,2.0
21,237895,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3093372,201.0,8.0,0.0
1,12,237883,0.0,1.0,0.0
2,15,1427592,57.0,57.0,0.0
3,16,951659,5.0,8.0,0.0
4,17,237924,1.0,0.0,0.0
5,18,951237,20.0,3.0,0.0
6,19,1427549,5.0,7.0,0.0
7,20,1665277,87.0,30.0,2.0
8,21,237895,0.0,1.0,0.0
9,22,475773,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,201.0,8.0,0.0
1,12,0.0,1.0,0.0
2,15,57.0,57.0,0.0
3,16,5.0,8.0,0.0
4,17,1.0,0.0,0.0
5,18,20.0,3.0,0.0
6,19,5.0,7.0,0.0
7,20,87.0,30.0,2.0
8,21,0.0,1.0,0.0
9,22,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,201.0
1,12,Released,0.0
2,15,Released,57.0
3,16,Released,5.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
Executing,8.0,1.0,57.0,8.0,0.0,3.0,7.0,30.0,1.0,2.0
Released,201.0,0.0,57.0,5.0,1.0,20.0,5.0,87.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,8.0,1.0,57.0,8.0,0.0,3.0,7.0,30.0,1.0,2.0
2,Released,201.0,0.0,57.0,5.0,1.0,20.0,5.0,87.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,8.0,1.0,57.0,8.0,0.0,3.0,7.0,30.0,1.0,2.0
2,Released,201.0,0.0,57.0,5.0,1.0,20.0,5.0,87.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()